In [1]:
%config IPCompleter.greedy=True
import dtapy.visualization
from settings import default_city as city
from network_data import load_pickle
import networkx as nx
import numpy as np
from core.time import SimulationTime
import importlib

ImportError: cannot import name 'data_folder' from '__init__' (C:\Workdir\Programs\research\dtapy\testing_dtapy\__init__.py)

In [8]:
importlib.reload(visualization)
g: nx.MultiDiGraph = load_pickle(city + '_grid_centroids')
g.number_of_edges()
time = SimulationTime(0,2,0.25)
flows = np.random.rand(time.tot_time_steps*g.number_of_edges()).reshape((time.tot_time_steps,g.number_of_edges()))*1200
costs = np.random.rand(time.tot_time_steps*g.number_of_edges()).reshape((time.tot_time_steps,g.number_of_edges()))*200

visualization.show_assignment(g, flows, costs, time)

Zinnowitz_grid_centroids.pickle not found in data folder!


TypeError: descriptor 'with_traceback' of 'BaseException' object needs an argument

In [3]:


from visualization import show_assignment

ImportError: cannot import name 'show_assignment' from 'visualization' (C:\Workdir\Programs\research\dtapy\testing_dtapy\visualization.py)